In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import json

In [2]:
#Reddit API credentials
CLIENT_ID = 'YOURCLIENTID'
CLIENT_SECRET = 'YOURCLIENTSECRET'
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'
USERNAME = 'username '
PASSWORD = 'password'


In [3]:
url = 'https://oauth.reddit.com/r/StopGaming/new'

In [4]:
# Create a session
session = requests.Session()

In [5]:
# Get the access token
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)

In [6]:
data = {
    'grant_type': 'password',
    'username': USERNAME,
    'password': PASSWORD
}
headers = {'User-Agent': USER_AGENT}

In [7]:
res = session.post('https://www.reddit.com/api/v1/access_token', auth=auth, data=data, headers=headers)
token = res.json()['access_token']
headers['Authorization'] = f'bearer {token}'

In [8]:
params = {'limit': 20}

In [9]:
res = session.get(url, headers=headers, params=params)

In [10]:
soup = BeautifulSoup(res.content)

In [11]:
json_data = json.loads(soup.find_all('p')[0].text)

In [12]:
posts = json_data['data']['children']

In [13]:
posts_data = []

In [14]:
for i in posts:
    data = i['data']
    title = data.get('title', 'No Title')
    date = datetime.fromtimestamp(data.get('created_utc', 0)).strftime('%Y-%m-%d %H:%M:%S')
    description = data.get('selftext', 'No Description')
    truncated_description = ' '.join(description.split()[:100])  # Truncate to 100 words
    upvotes = data.get('ups', 'No Upvotes')
    upvote_ratio = data.get('upvote_ratio', 'No Upvote Ratio')
    link = data.get('url', 'No URL')
    
    print(f"Title: {title}")
    print(f"Date: {date}")
    print(f"Description: {truncated_description}")
    print(f"Upvotes: {upvotes}")
    print(f"Upvote Ratio: {upvote_ratio}")
    print(f"Link: {link}")
    print("\n")
    
    posts_data.append({
        'Title': title,
        'Date Posted': date,
        'Description': truncated_description,
        'Upvotes': upvotes,
        'Upvote Ratio': upvote_ratio,
        'Link': link
    })

Title: How did you guys finally pull the plug? 
Date: 2024-07-18 09:43:53
Description: Everything in my mind tells me i need to pull it. Even when I try to think around it I still always end up back at the same conclusion. i am addicted. I'm slowing allowing myself to be killed right in front of my eyes, and yet I can't seem to do anything about it. So how did you do it, how did you pull the plug?
Upvotes: 2
Upvote Ratio: 1.0
Link: https://www.reddit.com/r/StopGaming/comments/1e627eu/how_did_you_guys_finally_pull_the_plug/


Title: 35 year old brother has been addicted to video games for years
Date: 2024-07-18 09:16:16
Description: I hope it's alright to post this here, I just wanted to ask for any advice or insight on how to approach this. My brother is 5 years older than me and has always enjoyed games, however since graduating college he has only applied for a couple of jobs (with the urging of our parents) that unfortunately didn't pan out, and hasn't worked ever since. My mom has 

In [15]:
posts_df = pd.DataFrame(posts_data)

In [16]:
posts_df.head()

,Title,Date Posted,Description,Upvotes,Upvote Ratio,Link
0,How did you guys finally pull the plug?,2024-07-18 09:43:53,Everything in my mind tells me i need to pull ...,2,1.00,https://www.reddit.com/r/StopGaming/comments/1...
1,35 year old brother has been addicted to video...,2024-07-18 09:16:16,"I hope it's alright to post this here, I just ...",1,1.00,https://www.reddit.com/r/StopGaming/comments/1...
2,Isn't gaming bad even if you are a successful ...,2024-07-18 05:56:21,I'm a 18 yo male and I'm basically in the midd...,4,0.83,https://www.reddit.com/r/StopGaming/comments/1...
3,4 months without gaming (especially Factorio!),2024-07-18 03:13:32,I am M30 and have been playing video games sin...,4,1.00,https://www.reddit.com/r/StopGaming/comments/1...
4,Gaming addiction has been controlling me for y...,2024-07-18 00:59:10,Hey guys. So I recently joined this subreddit ...,4,1.00,https://www.reddit.com/r/StopGaming/comments/1...


In [17]:
posts_df.to_csv('redditScrapedData.csv', index=False)